In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [2]:
def vocab():
    alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    special = [' ']
    vocab = alpha+special+['END']

    return vocab
    
def getIndex(df, maxlen, file):
    voc = vocab()
    char_index = dict((c, i) for i, c in enumerate(voc))
    
    X = []
    next_name = False
    
    # Club all the whitespaces
    
    trunc_X = [str(i).lower()[0:maxlen] for i in df['name']]
    mask = []
    for i in range(len(trunc_X)):
        name = trunc_X[i]
        tmp = []
        for char in str(name):
            if char not in voc:
                with open(file,'a') as f:
                    f.write(name+'\n')
                print(name)
                next_name = True
                break
            else:
                tmp.append(char_index[char])
        if next_name:
            mask.append(False)
            next_name = False
            continue
        mask.append(True)
        for k in range(0,maxlen - len(str(name))):
            tmp.append(char_index["END"])
        X.append(tmp)
            
    return np.asarray(X), df[mask]

In [10]:
import re
import numpy as np

def preprocess_df(df):
    df['name'] = [str(name).lower() for name in df['Name']]
    df['name'] = [str(name).strip() for name in df['name']]
    df['name'] = [re.sub('[^a-zA-Z]+', ' ', str(name)) for name in df['name']]
    df['name'] = [re.sub('[ \t\n]+',' ', str(name)) for name in df['name']]
    df['name'].replace('', np.nan, inplace=True)
    df = df[df['name']!='nan']
    df.dropna(subset=['name'],inplace=True)
    return df

In [39]:
def returnGenderPred(df, model):
    test_x, df = getIndex(df, 30, 'oov.txt')
    pred = model.predict(np.asarray(test_x), verbose = 1)
    y_pred = np.argmax(pred, axis =1)
    df['gender'] = y_pred
    return df

from keras.models import load_model


def get_gender_model():
    filename = "/home/souvic/mounted/btp/vahini/Models/ERTrain/SavedModels/FinalSet/CharLSTM/ERModel/ERModel"
    model = load_model(filename)
    return model

def get_gender_model2():
    filename = "/home/souvic/mounted/btp/vahini/Name2Demographics/Models/ERData/CharCNN_CharLSTM/SavedModel/ERModel"
    model = load_model(filename)
    return model

def get_gender_model3():
    filename = "/home/souvic/mounted/btp/vahini/Name2Demographics/Models/CBSEData/CharCNN/SavedModel/CharCNN"
    model = load_model(filename)
    return model


In [40]:
gender_model = get_gender_model()
gender_model2 = get_gender_model2()
gender_model3 = get_gender_model3()

In [7]:
%ls

'Add Labels.ipynb'   Baselines/        er_sample.csv
 aieee_sample.csv    cbse_sample.csv


16/16 [==============================] - 2s 116ms/step


In [41]:
import pandas as pd
import numpy as np

df = pd.read_csv("er_sample.csv")
df = preprocess_df(df)
df = returnGenderPred(df, gender_model)

df['gender'] = [str(elem).replace("0","Male") for elem in df['gender']]
df['gender'] = [str(elem).replace("1","female") for elem in df['gender']]


count = 0
for index, row in df.iterrows():
    if(str(row['Gender']).strip().lower() == str(row['gender']).strip().lower()):
        count = count + 1
#         print(row['Gender'], row['gender'])


        
total = df.shape[0]
accuracy = round(count/total * 100, 2)
print(accuracy)

16/16 [==============================] - 0s 3ms/step
73.65


In [42]:
import pandas as pd
import numpy as np

df = pd.read_csv("aieee_sample.csv")
df = preprocess_df(df)
df = returnGenderPred(df, gender_model3)

df['gender'] = [str(elem).replace("0","Male") for elem in df['gender']]
df['gender'] = [str(elem).replace("1","female") for elem in df['gender']]


count = 0
for index, row in df.iterrows():
    if(str(row['Gender']).strip().lower() == str(row['gender']).strip().lower()):
        count = count + 1
#         print(row['Gender'], row['gender'])

        
total = df.shape[0]
accuracy = round(count/total * 100, 2)
print(accuracy)


16/16 [==============================] - 0s 3ms/step
85.6


In [43]:
import pandas as pd
import numpy as np

df = pd.read_csv("cbse_sample.csv")
df = preprocess_df(df)
df = returnGenderPred(df, gender_model3)

df['gender'] = [str(elem).replace("0","Male") for elem in df['gender']]
df['gender'] = [str(elem).replace("1","female") for elem in df['gender']]


count = 0
for index, row in df.iterrows():
    if(str(row['Gender']).strip().lower() == str(row['gender']).strip().lower()):
        count = count + 1
#         print(row['Gender'], row['gender'])

        
total = df.shape[0]
accuracy = round(count/total * 100, 2)
print(accuracy)


16/16 [==============================] - 0s 2ms/step
97.2


In [86]:
path = "/home/souvic/mounted/btp/ndata/samples/Baselines/Forebears/forebears_er_first_name_part1.csv"
path1 = "/home/souvic/mounted/btp/ndata/samples/Baselines/Forebears/forebears_er_first_name_part2.csv"


import pandas as pd

df  = pd.read_csv(path)
df1 = pd.read_csv(path1)

total_df = pd.concat([df, df1], ignore_index=True)
fb_df = pd.DataFrame()
fb_df['Name']= total_df['Name']
fb_df['first_name'] = total_df['first_name']
fb_df['fb_gender'] = total_df['world_gender']
fb_df['Gender'] = total_df['Gender']

import pandas as pd
import numpy as np

er_df = pd.read_csv("er_sample.csv")
er_df = preprocess_df(er_df)
er_df = returnGenderPred(er_df, gender_model)

our_df = pd.DataFrame()
our_df['Name'] = er_df['Name']
our_df['new_gender']= er_df['gender']
our_df['new_gender'] = [str(elem).replace("0","Male") for elem in our_df['new_gender']]
our_df['new_gender'] = [str(elem).replace("1","female") for elem in our_df['new_gender']]

16/16 [==============================] - 2s 124ms/step


In [87]:
mixed_df = fb_df.merge(our_df, on="Name", how="left")

In [88]:
#  Get a df on er where data fails
fail_fem_df = mixed_df[(mixed_df['fb_gender']=='m') & (mixed_df['Gender']=='Female') & (mixed_df['new_gender']=='female') ]
fail_male_df = mixed_df[(mixed_df['fb_gender']=='f') & (mixed_df['Gender']=='Male') & (mixed_df['new_gender']=='Male') ]


In [89]:
fail_model_df = mixed_df[((mixed_df['Gender']=='Female') & (mixed_df['new_gender']=='Male')) | ((mixed_df['Gender']=='Male') & (mixed_df['new_gender']=='female'))]
                        
                        

In [90]:
fail_fem_df

,Name,first_name,fb_gender,Gender,new_gender
9,hijam dayapati devi,hijam,m,Female,female
13,ch. khangamchon,ch.,m,Female,female
20,k p. rathingla,k,m,Female,female
23,rs kumtharla,rs,m,Female,female
24,arambam sanathokpi,arambam,m,Female,female
26,chingakham romita,chingakham,m,Female,female
29,hijam ratna chanu,hijam,m,Female,female
37,laishram priyabati,laishram,m,Female,female
46,pheiroijam shila,pheiroijam,m,Female,female
49,sa. leiyachan,sa.,m,Female,female


In [91]:
fail_male_df

,Name,first_name,fb_gender,Gender,new_gender
1,dhodi dineshbhai babubhai,dhodi,f,Male,Male
7,khumukcham milanchandra,khumukcham,f,Male,Male
21,kumam kenion singh,kumam,f,Male,Male
27,t thuamkhanson,t,f,Male,Male
50,khumukcham newtonkumar,khumukcham,f,Male,Male
51,konjengbam suresh meitei,konjengbam,f,Male,Male
57,sinam manglemjao,sinam,f,Male,Male
66,kangabam hemanta,kangabam,f,Male,Male
91,robin mawblei,robin,f,Male,Male
112,embha lyngdoh,embha,f,Male,Male


In [92]:
fail_model_df

,Name,first_name,fb_gender,Gender,new_gender
55,k. ngaipi,k.,m,Male,female
63,moirangthem shubha,moirangthem,m,Female,Male
84,turim,turim,f,Female,Male
89,other's : lina sun,other's,NaN,Male,female
97,mingrose nongkseh,mingrose,m,Female,Male
99,other's : sailetson marak,other's,NaN,Female,Male
122,gretchilda marak,gretchilda,f,Male,female
124,chaoba gondaime!,chaoba,m,Male,female
128,jmod kyrsian,jmod,f,Female,Male
136,iarbi talang,iarbi,f,Male,female


In [82]:
path = "/home/souvic/mounted/btp/ndata/samples/Baselines/Forebears/forebears_aieee_first_name_part1.csv"
path1 = "/home/souvic/mounted/btp/ndata/samples/Baselines/Forebears/forebears_aieee_first_name_part2.csv"


import pandas as pd

df  = pd.read_csv(path)
df1 = pd.read_csv(path1)

total_df = pd.concat([df, df1], ignore_index=True)
fb_df = pd.DataFrame()
fb_df['Name']= total_df['Name']
fb_df['first_name'] = total_df['first_name']
fb_df['fb_gender'] = total_df['world_gender']
fb_df['Gender'] = total_df['Gender']

import pandas as pd
import numpy as np

er_df = pd.read_csv("aieee_sample.csv")
er_df = preprocess_df(er_df)
er_df = returnGenderPred(er_df, gender_model)

our_df = pd.DataFrame()
our_df['Name'] = er_df['Name']
our_df['new_gender']= er_df['gender']
our_df['new_gender'] = [str(elem).replace("0","Male") for elem in our_df['new_gender']]
our_df['new_gender'] = [str(elem).replace("1","female") for elem in our_df['new_gender']]

mixed_df = fb_df.merge(our_df, on="Name", how="left")
#  Get a df on er where data fails
fail_fem_df = mixed_df[(mixed_df['fb_gender']=='m') & (mixed_df['Gender']=='Female') & (mixed_df['new_gender']=='female') ]
fail_male_df = mixed_df[(mixed_df['fb_gender']=='f') & (mixed_df['Gender']=='Male') & (mixed_df['new_gender']=='Male') ]

fail_model_df = mixed_df[((mixed_df['Gender']=='Female') & (mixed_df['new_gender']=='Male')) | ((mixed_df['Gender']=='Male') & (mixed_df['new_gender']=='female'))]
                        
                        


16/16 [==============================] - 2s 125ms/step


In [85]:
fail_male_df

,Name,first_name,fb_gender,Gender,new_gender
34,kosuru nageswararao,kosuru,f,Male,Male
70,b.venkateshwarulu,b.venkateshwarulu,f,Male,Male
76,v.srinivas,v.srinivas,f,Male,Male
87,joy jose,joy,f,Male,Male
122,v jacob philip,v,f,Male,Male
168,vimlesh kumar sharma,vimlesh,f,Male,Male
